# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [60]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [73]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [62]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [63]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [64]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [65]:
#Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkify 
                WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor': 1} 
                """)

#### Set Keyspace

In [66]:
#Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




Query #1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In order to answer to this question, we need to select the artist name, song name, and song length from out table and we will filter by sessionId and itemInSession.In CQL this query will look in the following way:

SELECT artist, song_title, song_length FROM songs_session WHERE sessionId = 338 AND itemInSession = 4

1. We'll name our table songs_session;
2. Our primary key will consist of partition key sessionId, and clustering key itemInSession in this way we can filter by these attributes in other later steps;
3. Columns of the table will be: sessionId, itemInSession, artist, song_title and song_length.

In [67]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
session.execute("""
    CREATE TABLE IF NOT EXISTS songs_session(
          sessionId int, 
          itemInSession int, 
          artist text, 
          song_title text, 
          song_length float,
          PRIMARY KEY(sessionId, itemInSession)
          )
    """)

                    

In [68]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(sessionId), int(itemInSession), artist_name, song, float(length)))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [69]:
rows = session.execute("""SELECT  artist, song_title, song_length FROM songs_session WHERE sessionId = 338 and itemInSession = 4""")

#collect all rows in a tuple
allobs=[]
for row in rows:
    allobs.append((row.artist, row.song_title, row.song_length))
    
#Create DataFrame from the output query
df=pd.DataFrame(allobs, columns=['artist', 'song_title', 'song_length'])

#print the result
df.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Query #2

Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query we'll select artist name, song name, user name and user lastname from source table, we will need to filter by userId and sessionId, and order by itemInSession.

In CQL will have the following representation:

SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182

Steps:
1. Name of our table songs_user
2. Primary key will be a composite partition key userId, sessionId. The reason of choosing a composite partition key is to drive scalability to our data model, using only userId will place in different nodes sessions which belongs to the same userId (in case the data volume will increase there will be performance issues in querying in this data model if we would choose to use only userId in our key)
3. Our cluster key will be itemInSession, so will order by this attribute;
4. Columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.

In [70]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
session.execute("""
     CREATE TABLE IF NOT EXISTS songs_user(
          userId int, 
          sessionId int, 
          itemInSession int, 
          artist text, 
          song text, 
          firstName text, 
          lastName text,
          PRIMARY KEY((userId, sessionId), itemInSession)
    )
""")


                    

In [71]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_user (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

In [72]:
rows = session.execute("""SELECT  itemInSession, artist, song, firstName, lastName FROM songs_user WHERE userId = 10 AND sessionId = 182""")

#collect all rows in a tuple
allobs=[]
for row in rows:
    allobs.append((row.iteminsession, row.artist, row.song, row.firstname, row.lastname))

#Create DataFrame from the output query
df1=pd.DataFrame(allobs, columns=['iteminsession', 'artist', 'song', 'firstName', 'lastName'])

#print the result
df1.head()

,iteminsession,artist,song,firstName,lastName
0,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,1,Three Drives,Greece 2000,Sylvie,Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


Query #3
Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query we'll select the user first name and lastname from out table, and we will need to filter by song name. Due to the fact user name and lastname in large datasets are not unique, we'll add the attribute column userId in order to identify users uniquely.

In CQL the representation of our query will be the following:

SELECT firstName, lastName FROM lib_app_history WHERE song = 'All Hands Against His Own'

Steps:
1. The name of our table is lib_app_history
2. Our primary key will consist of partition key song, and clustering key userId. This identifies our rows uniquely.
3. Columns of our table will be: song, firstName, lastName and userId.

In [52]:

session.execute("""
        CREATE TABLE IF NOT EXISTS lib_app_history(
            song text, 
            firstName text, 
            lastName text,
            userId int, 
            PRIMARY KEY(song, userId) 
    )
""")

                    

In [53]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO lib_app_history (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, firstName, lastName, int(userId)))

In [54]:
rows = session.execute("""SELECT firstName, lastName FROM lib_app_history WHERE song= 'All Hands Against His Own'""")

#collect all rows in a tuple
allobs=[]
for row in rows:
    allobs.append((row.firstname, row.lastname))
  
#Create DataFrame from the output query
df2=pd.DataFrame(allobs, columns=['firstName', 'lastName'])

#print the result
df2.head()

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [58]:
## TO-DO: Drop the table before closing out the sessions


session.execute("""DROP TABLE songs_session""")
session.execute("""DROP TABLE songs_user""")
session.execute("""DROP TABLE lib_app_history""")

### Close the session and cluster connection¶

In [59]:
session.shutdown()
cluster.shutdown()